In [2]:
import requests

url = "https://www.dropbox.com/s/y4ss9gmw6ykde51/twitter-melb.json.tar.gz"
filename = url.split("/")[-1]
with open(filename, "wb") as f:
    r = requests.get(url)
    f.write(r.content)

In [6]:
import gzip
import urllib.request

def download_file(url):
    out_file = '/Users/weimin/Downloads/CCC/CCC_A2/Twitter-Harvester/twitter-melb.json'

    # Download archive
    try:
        # Read the file inside the .gz archive located at url
        with urllib.request.urlopen(url) as response:
            with gzip.GzipFile(fileobj=response) as uncompressed:
                file_content = uncompressed.read()

          # write to file in binary mode 'wb'
        with open(out_file, 'wb') as f:
            f.write(file_content)
            return 0

    except Exception as e:
        print(e)
        return 1

retval=download_file('https://www.dropbox.com/s/y4ss9gmw6ykde51/twitter-melb.json.tar.gz') 

Not a gzipped file (b'<!')


In [1]:
import os
import json
twitter_path = 'twitter-melb.json'
filesize = os.path.getsize(twitter_path)
print(filesize)

chunksize = filesize / 100

FileNotFoundError: [Errno 2] No such file or directory: 'twitter-melb.json'

In [2]:
def chunk(twitter_path, chunksize, filesize):
    
    with open(twitter_path, 'rb') as f:
        # Get current position in file
        chunk_end = f.tell()

        while True:
            chunk_start = chunk_end
            # Move from current position to current position + chunk_size
            f.seek(f.tell() + chunksize)
            # Read until the next line to not break lines in the middle
            f.readline()
            # Get chunk_end as the offset from the beginning of the file in bytes
            chunk_end = f.tell()
            if chunk_end > filesize:
                chunk_end = filesize
            yield chunk_start, chunk_end - chunk_start
            if chunk_end == filesize:
                break

def read_tweets(twitter_path, start, size):
    """
    param: twitter file path
    return: twitter file lines
    """

    with open(twitter_path, 'rb') as f:
        # Get current position in file
        f.seek(start)
        end = start
        while True:
            line = f.readline()
            end = f.tell()
            yield line
            if end > start + size:
                end = start + size
            if end == start + size:
                break
            
def proc_line(line):
    """
    param: twitter file lines
    return: processed twitter file lines
    """
    line = line.decode('utf-8')
    line = line.strip('\r\n')
    line = line.rstrip('')
    if line.endswith (','):
        line = line[:-1]
    if line.endswith (']}'): 
        line = line[:-2]

    return line

In [3]:
# couchbd_settings
address = 'localhost:15984'
username = 'admin'
password = '123456'
tweets = 'raw_tweets'
test = 'test'

In [4]:
import couchdb

# use authenticated connection to connect to couchdb server for writing to the database
def db_connect(dbname):

    couchserver = couchdb.Server('http://' + username + ':' + password + '@' + address)
    try:
      db = couchserver[dbname]
    except:
      db = couchserver.create(dbname)

    return db

test_db = db_connect(test)
tweet_db = db_connect(tweets)

def save_to_db(item_id, keyword, item, db):
    """"""
    """"""

    db[item_id] = {keyword: item}

In [5]:
chunks = []

for chunkStart, chunkSize in chunk(twitter_path, int(chunksize), filesize):
            chunks.append({'chunkStart': chunkStart, 'chunkSize': chunkSize})

In [9]:
import itertools

chunkStart = chunks[9]['chunkStart']
chunkSize = chunks[9]['chunkSize']
yield_tweets = read_tweets(twitter_path,  chunkStart, chunkSize)

for line in yield_tweets:
    line_processed = proc_line(line)
    try:
        data = json.loads(line_processed)
        tweet_id = data['id']
        if tweet_id not in tweet_db and tweet_id not in test_db:
            save_to_db(tweet_id, 'historic', data['doc'], test_db)
    except Exception as e:
        print('Error reading line --ignoring')
        print(e)      
        